In [20]:
import os 
import pytesseract
from transformers import LayoutLMv3Processor, LayoutLMForTokenClassification
from PIL import Image 
import torch
import json

In [21]:
modelo = "microsoft/layoutlmv3-base"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [22]:
processor = LayoutLMv3Processor.from_pretrained(modelo, apply_ocr = False)
model = LayoutLMForTokenClassification.from_pretrained(modelo).to(device)

You are using a model of type layoutlmv3 to instantiate a model of type layoutlm. This is not supported for all configurations of models and can yield errors.
Some weights of LayoutLMForTokenClassification were not initialized from the model checkpoint at microsoft/layoutlmv3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'layoutlm.embeddings.LayerNorm.bias', 'layoutlm.embeddings.LayerNorm.weight', 'layoutlm.embeddings.h_position_embeddings.weight', 'layoutlm.embeddings.position_embeddings.weight', 'layoutlm.embeddings.token_type_embeddings.weight', 'layoutlm.embeddings.w_position_embeddings.weight', 'layoutlm.embeddings.word_embeddings.weight', 'layoutlm.embeddings.x_position_embeddings.weight', 'layoutlm.embeddings.y_position_embeddings.weight', 'layoutlm.encoder.layer.0.attention.output.LayerNorm.bias', 'layoutlm.encoder.layer.0.attention.output.LayerNorm.weight', 'layoutlm.encoder.layer.0.attention.output.dense.bias', 'layoutlm.encoder.layer.0.attention.ou

In [23]:
ruta_imagenes = "/home/pibezx/Documents/Proyectos/PaginaWeb_Automoviles/Chatbot_Cars/imagenes_fichas/"
ruta_salida = "/home/pibezx/Documents/Proyectos/PaginaWeb_Automoviles/Chatbot_Cars/datos_extraidos"

In [24]:
def extraer_datos(imagen_path):
    try:
        imagen = Image.open(imagen_path).convert("RGB")
        
        # Usamos pytesseract para OCR
        palabras = pytesseract.image_to_data(imagen, lang='spa', output_type=pytesseract.Output.DICT)
        
        # Filtrar solo las palabras no vacías
        palabras_filtradas = []
        boxes_filtrados = []
        
        for i in range(len(palabras['text'])):
            if palabras['text'][i].strip() != "":
                palabras_filtradas.append(palabras['text'][i])
                # Convertir a formato esperado por LayoutLMv3: [x_izq, y_sup, x_der, y_inf]
                x_izq = palabras['left'][i]
                y_sup = palabras['top'][i]
                x_der = x_izq + palabras['width'][i]
                y_inf = y_sup + palabras['height'][i]
                boxes_filtrados.append([x_izq, y_sup, x_der, y_inf])
        
        if not palabras_filtradas:
            print(f"No se encontró texto en {imagen_path}")
            return {"error": "No se encontró texto en la imagen"}
        
        # Procesamiento con LayoutLMv3
        encoding = processor(
            imagen, 
            words=palabras_filtradas,  # Usar palabras en lugar de texto
            boxes=boxes_filtrados,  # Usar cajas en formato correcto
            return_tensors="pt"
        )
        
        encoding = {k:v.to(device) for k,v in encoding.items()}
        
        # Inferencia del modelo
        with torch.no_grad():
            outputs = model(**encoding)
            
        logits = outputs.logits
        predictions = logits.argmax(-1).squeeze().tolist()
        
        # Asegurarse de que predictions es una lista
        if not isinstance(predictions, list):
            predictions = [predictions]
        
        # Procesar las predicciones
        resultado = {}
        token_ids = encoding['input_ids'][0].tolist()
        tokens = processor.tokenizer.convert_ids_to_tokens(token_ids)
        
        # Mapear tokens a palabras originales
        palabra_actual = ""
        etiqueta_actual = ""
        for token, pred_id in zip(tokens, predictions):
            # Ignorar tokens especiales como [CLS], [SEP], etc.
            if token.startswith("##"):
                palabra_actual += token[2:]
            elif token in processor.tokenizer.all_special_tokens:
                continue
            else:
                if palabra_actual and etiqueta_actual != "O":
                    if etiqueta_actual not in resultado:
                        resultado[etiqueta_actual] = palabra_actual
                    else:
                        resultado[etiqueta_actual] += f" {palabra_actual}"
                
                palabra_actual = token
                etiqueta_actual = model.config.id2label.get(pred_id, "O")
        
        # No olvidar la última palabra
        if palabra_actual and etiqueta_actual != "O":
            if etiqueta_actual not in resultado:
                resultado[etiqueta_actual] = palabra_actual
            else:
                resultado[etiqueta_actual] += f" {palabra_actual}"
        
        return resultado
    
    except Exception as e:
        print(f"Error procesando {imagen_path}: {str(e)}")
        return {"error": str(e)}

In [25]:
datos_todas_fichas = {}
imagenes_folder = "/home/pibezx/Documents/Proyectos/PaginaWeb_Automoviles/Chatbot_Cars/imagenes_fichas/"
for img_file in os.listdir(imagenes_folder):
    if img_file.lower().endswith(".png"):
        ruta_img = os.path.join(imagenes_folder, img_file)
        print(f"Procesando {ruta_img}")
        datos = extraer_datos(ruta_img)
        datos["imagen_fuente"] = img_file

        # Guarda el resultado en JSON
        with open(f"datos_extraidos/{img_file}.json", "w", encoding="utf-8") as f:
            json.dump(datos, f, ensure_ascii=False, indent=4)

Procesando /home/pibezx/Documents/Proyectos/PaginaWeb_Automoviles/Chatbot_Cars/imagenes_fichas/202411-02-onix-sedan-turbo-my25_297.png
Error procesando /home/pibezx/Documents/Proyectos/PaginaWeb_Automoviles/Chatbot_Cars/imagenes_fichas/202411-02-onix-sedan-turbo-my25_297.png: 'words'
Procesando /home/pibezx/Documents/Proyectos/PaginaWeb_Automoviles/Chatbot_Cars/imagenes_fichas/CATALOGO_RAV4_PERU_0_180.png
Error procesando /home/pibezx/Documents/Proyectos/PaginaWeb_Automoviles/Chatbot_Cars/imagenes_fichas/CATALOGO_RAV4_PERU_0_180.png: 'words'
Procesando /home/pibezx/Documents/Proyectos/PaginaWeb_Automoviles/Chatbot_Cars/imagenes_fichas/CATALOGO_4RUNNER_PERU_0_100.png
Error procesando /home/pibezx/Documents/Proyectos/PaginaWeb_Automoviles/Chatbot_Cars/imagenes_fichas/CATALOGO_4RUNNER_PERU_0_100.png: 'words'
Procesando /home/pibezx/Documents/Proyectos/PaginaWeb_Automoviles/Chatbot_Cars/imagenes_fichas/ficha-tecnica_suzuki-pe_s-presso_555.png
Error procesando /home/pibezx/Documents/Proyect

KeyboardInterrupt: 